In [15]:
import re
import numpy as np  
import pandas as pd 

from collections import Counter

from sklearn.preprocessing import LabelEncoder
from sklearn.utils import column_or_1d

In [16]:
data = pd.read_csv('../Data/cleaned_invalsi.csv', sep=",", engine="python").drop('Unnamed: 0', axis = 1)

In [17]:
data.head()

,cod_scu_anonimo,cod_cla_anonimo,cod_stu_anonimo,macrotipologia,sesso,mese,anno,luogo,eta,freq_asilo_nido,...,num_imm1,class_imm1,prop_imm1,num_imm2,class_imm2,prop_imm2,num_late,class_late,prop_late,public
0,29587,28127,20,ISTRUZIONE I CICLO,Maschio,Ottobre,2000,Italia (o,NaN,NaN,...,NaN,0,0.0,NaN,0,0.0,NaN,2,0.095238,1
1,29587,28127,10,ISTRUZIONE I CICLO,Maschio,Dicembre,2000,Italia (o,NaN,NaN,...,NaN,0,0.0,NaN,0,0.0,NaN,2,0.095238,1
2,29587,28127,4,ISTRUZIONE I CICLO,Femmina,Ottobre,2000,Italia (o,NaN,NaN,...,NaN,0,0.0,NaN,0,0.0,NaN,2,0.095238,1
3,29587,28127,9,ISTRUZIONE I CICLO,Femmina,Giugno,2000,Italia (o,NaN,NaN,...,NaN,0,0.0,NaN,0,0.0,NaN,2,0.095238,1
4,29587,28127,6,ISTRUZIONE I CICLO,Maschio,Giugno,2000,Italia (o,NaN,NaN,...,NaN,0,0.0,NaN,0,0.0,NaN,2,0.095238,1


In [18]:
data1 = data[(data.cod_scu_anonimo != 30005)&(data.cod_scu_anonimo != 30927)] # these schools have contradictory fields

In [19]:
data1['improv_mate_rapp'] = data1['mate8']/data1['mate6']
data1['improv_mate_diff'] = data1['mate8']-data1['mate6']
data1['improv_ita_rapp'] = data1['ita8']/data1['ita6']
data1['improv_ita_diff'] = data1['ita8']-data1['ita6']

<ipython-input-19-af9e89ebc440>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['improv_mate_rapp'] = data1['mate8']/data1['mate6']
<ipython-input-19-af9e89ebc440>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['improv_mate_diff'] = data1['mate8']-data1['mate6']
<ipython-input-19-af9e89ebc440>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

In [22]:
datax = pd.DataFrame()
datax['prop_maschi'] = pd.crosstab(data1.cod_scu_anonimo,data1.sesso)['Maschio']/pd.crosstab(data1.cod_scu_anonimo,data1.sesso).sum(axis=1)
datax['prop_laurea_padre'] = pd.crosstab(data1.cod_scu_anonimo,data1.titolo_padre)['Laurea']/pd.crosstab(data1.cod_scu_anonimo,data1.fillna('@').titolo_padre).sum(axis=1)
datax['prop_laurea_madre'] = pd.crosstab(data1.cod_scu_anonimo,data1.titolo_madre)['Laurea']/pd.crosstab(data1.cod_scu_anonimo,data1.fillna('@').titolo_madre).sum(axis=1)
datax['prop_diploma_padre'] = pd.crosstab(data1.cod_scu_anonimo,data1.titolo_padre)['Diploma']/pd.crosstab(data1.cod_scu_anonimo,data1.fillna('@').titolo_padre).sum(axis=1)
datax['prop_diploma_madre'] = pd.crosstab(data1.cod_scu_anonimo,data1.titolo_madre)['Diploma']/pd.crosstab(data1.cod_scu_anonimo,data1.fillna('@').titolo_madre).sum(axis=1)
datax['prop_padre_ita'] = pd.crosstab(data1.cod_scu_anonimo,data1.luogo_padre)['Italia (o']/pd.crosstab(data1.cod_scu_anonimo,data1.fillna('@').luogo_padre).sum(axis=1)
datax['prop_madre_ita'] = pd.crosstab(data1.cod_scu_anonimo,data1.luogo_madre)['Italia (o']/pd.crosstab(data1.cod_scu_anonimo,data1.fillna('@').luogo_madre).sum(axis=1)
datax['prop_padre_disocc'] = pd.crosstab(data1.cod_scu_anonimo,data1.prof_padre)['Disoccu']/pd.crosstab(data1.cod_scu_anonimo,data1.fillna('@').prof_padre).sum(axis=1)
datax['prop_madre_disocc'] = pd.crosstab(data1.cod_scu_anonimo,data1.prof_madre)['Disoccu']/pd.crosstab(data1.cod_scu_anonimo,data1.fillna('@').prof_madre).sum(axis=1)
datax.loc[30012, 'prop_padre_disocc'] = np.nan
datax.loc[30012, 'prop_madre_disocc'] = np.nan
datax['prop_bocciati'] = pd.crosstab(data1.cod_scu_anonimo,data1['regolarita\''])['Posticipat']/pd.crosstab(data1.cod_scu_anonimo,data1['regolarita\'']).sum(axis=1)

datax

,prop_maschi,prop_laurea_padre,prop_laurea_madre,prop_diploma_padre,prop_diploma_madre,prop_padre_ita,prop_madre_ita,prop_padre_disocc,prop_madre_disocc,prop_bocciati
cod_scu_anonimo,,,,,,,,,,
29587,0.619048,0.047619,0.000000,0.000000,0.047619,0.952381,0.904762,0.000000,0.000000,0.095238
29588,0.538462,0.307692,0.307692,0.307692,0.307692,0.923077,1.000000,0.000000,0.076923,0.076923
29589,0.600000,0.000000,0.100000,0.350000,0.350000,1.000000,0.800000,0.000000,0.150000,0.000000
29590,0.529412,0.058824,0.117647,0.235294,0.117647,1.000000,0.882353,0.000000,0.000000,0.000000
29591,0.466667,0.000000,0.133333,0.200000,0.333333,0.800000,0.866667,0.000000,0.000000,0.133333
...,...,...,...,...,...,...,...,...,...,...
30996,0.500000,0.050000,0.100000,0.550000,0.600000,0.800000,0.850000,0.050000,0.000000,0.050000
30997,0.578947,0.052632,0.052632,0.263158,0.421053,0.947368,0.894737,0.000000,0.000000,0.052632
30998,0.400000,0.080000,0.160000,0.240000,0.320000,0.800000,0.760000,0.040000,0.120000,0.160000


In [23]:
datax.mean()

prop_maschi           0.496437
prop_laurea_padre     0.102690
prop_laurea_madre     0.111948
prop_diploma_padre    0.291382
prop_diploma_madre    0.341646
prop_padre_ita        0.847252
prop_madre_ita        0.837218
prop_padre_disocc     0.042970
prop_madre_disocc     0.033513
prop_bocciati         0.094038
dtype: float64

In [24]:
# To drop
gdrop = ['cod_cla_anonimo','cod_stu_anonimo','macrotipologia']
data2 = data1.drop(gdrop, axis = 1)
    
# To group with first
num_first = [d for d in data1.columns]
gfirst = [c for c in data1.columns if len(data1[data1.cod_scu_anonimo == 30996][c].unique()) == 1 and c not in gdrop+['cod_scu_anonimo']]
data3 = data2.loc[:,['cod_scu_anonimo']+gfirst].groupby(by  ='cod_scu_anonimo').first()
    
# To group with functions (mean, std, skew, quantiles...)
gmean = [d for d in data2.select_dtypes(np.number).columns if d not in gfirst+['cod_scu_anonimo']]

def q25(x):
    return np.quantile(x,0.25)
def q75(x):
    return np.quantile(x,0.75)
    
data4 = data2.loc[:, ['cod_scu_anonimo']+gmean].groupby(by = 'cod_scu_anonimo').agg(['mean','std','skew','min','max','median',q25,q75])

group_data = pd.concat([data3,data4],axis = 1)
group_data.head()

,freq_scuola_materna,voto_scritto_ita,voto_scritto_mat,voto_scritto_ita_cont,voto_scritto_mat_cont,prov,Nome_reg,Pon,Fattore_correzione,D1_a,...,"(improv_ita_rapp, q25)","(improv_ita_rapp, q75)","(improv_ita_diff, mean)","(improv_ita_diff, std)","(improv_ita_diff, skew)","(improv_ita_diff, min)","(improv_ita_diff, max)","(improv_ita_diff, median)","(improv_ita_diff, q25)","(improv_ita_diff, q75)"
cod_scu_anonimo,,,,,,,,,,,,,,,,,,,,,
29587,No,NaN,NaN,NaN,NaN,PZ,Basilicata,Area_non_P,0.987523,Positivo,...,NaN,NaN,3.276051,17.450286,1.665511,-16.398922,53.410194,0.701176,NaN,NaN
29588,SÃ¬,8.0,9.0,8.0,9.0,CB,Molise,Area_non_P,0.998281,NaN,...,NaN,NaN,7.168104,11.629300,0.195580,-10.188904,25.618491,5.944374,NaN,NaN
29589,SÃ¬,9.0,9.0,9.0,9.0,CB,Molise,Area_non_P,0.986064,Positivo,...,NaN,NaN,5.819583,21.114012,0.308100,-48.067528,66.698949,4.216302,NaN,NaN
29590,SÃ¬,7.0,5.0,NaN,NaN,TN,Prov. Aut. Trento,Area_non_P,0.993546,Positivo,...,0.941879,1.026787,0.239267,10.706817,1.771093,-16.307340,33.408610,-2.598044,-4.981769,2.066405
29591,SÃ¬,NaN,NaN,NaN,NaN,TN,Prov. Aut. Trento,Area_non_P,0.983432,NaN,...,NaN,NaN,-12.647059,11.139396,-0.003021,-34.802655,10.092763,-11.095005,NaN,NaN


In [25]:
gpp = group_data.join(datax)
gpp

,freq_scuola_materna,voto_scritto_ita,voto_scritto_mat,voto_scritto_ita_cont,voto_scritto_mat_cont,prov,Nome_reg,Pon,Fattore_correzione,D1_a,...,prop_maschi,prop_laurea_padre,prop_laurea_madre,prop_diploma_padre,prop_diploma_madre,prop_padre_ita,prop_madre_ita,prop_padre_disocc,prop_madre_disocc,prop_bocciati
cod_scu_anonimo,,,,,,,,,,,,,,,,,,,,,
29587,No,NaN,NaN,NaN,NaN,PZ,Basilicata,Area_non_P,0.987523,Positivo,...,0.619048,0.047619,0.000000,0.000000,0.047619,0.952381,0.904762,0.000000,0.000000,0.095238
29588,SÃ¬,8.0,9.0,8.0,9.0,CB,Molise,Area_non_P,0.998281,NaN,...,0.538462,0.307692,0.307692,0.307692,0.307692,0.923077,1.000000,0.000000,0.076923,0.076923
29589,SÃ¬,9.0,9.0,9.0,9.0,CB,Molise,Area_non_P,0.986064,Positivo,...,0.600000,0.000000,0.100000,0.350000,0.350000,1.000000,0.800000,0.000000,0.150000,0.000000
29590,SÃ¬,7.0,5.0,NaN,NaN,TN,Prov. Aut. Trento,Area_non_P,0.993546,Positivo,...,0.529412,0.058824,0.117647,0.235294,0.117647,1.000000,0.882353,0.000000,0.000000,0.000000
29591,SÃ¬,NaN,NaN,NaN,NaN,TN,Prov. Aut. Trento,Area_non_P,0.983432,NaN,...,0.466667,0.000000,0.133333,0.200000,0.333333,0.800000,0.866667,0.000000,0.000000,0.133333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30996,SÃ¬,NaN,NaN,NaN,NaN,CN,Piemonte,Area_non_P,0.996134,Parzialmen,...,0.500000,0.050000,0.100000,0.550000,0.600000,0.800000,0.850000,0.050000,0.000000,0.050000
30997,SÃ¬,7.0,6.0,7.0,6.0,CN,Piemonte,Area_non_P,0.981022,Positivo,...,0.578947,0.052632,0.052632,0.263158,0.421053,0.947368,0.894737,0.000000,0.000000,0.052632
30998,SÃ¬,NaN,NaN,NaN,NaN,AT,Piemonte,Area_non_P,0.972717,Positivo,...,0.400000,0.080000,0.160000,0.240000,0.320000,0.800000,0.760000,0.040000,0.120000,0.160000


In [26]:
list(gpp.columns)
todrop = ['freq_scuola_materna']
gpp = gpp.drop(todrop,axis=1)

In [27]:
# regional subdivision
sud = ['Basilicata', 'Molise','Sardegna', 'Sicilia', 'Calabria', 'Puglia', 'Campania', 'Abruzzo']
centro = ['Lazio', 'Marche', 'Umbria', 'Toscana', 'Emilia-Romagna']
nord = ['Prov. Aut. Trento', 'Liguria', 'Piemonte','Friuli-Venezia Giulia', 'Veneto', 'Prov. Aut. Bolzano (l. it.)','Lombardia', "Valle d'Aosta"]
for index,row in gpp.iterrows():
    gpp.loc[index,'Sud'] = 0
    gpp.loc[index,'Centro'] = 0
    gpp.loc[index,'Nord'] = 0
    if row['Nome_reg'] in sud:
        gpp.loc[index,'Sud'] = 1
    elif row['Nome_reg'] in centro:
        gpp.loc[index,'Centro'] = 1
    elif row['Nome_reg'] in nord:
        gpp.loc[index,'Nord'] = 1    

In [28]:
gpp.to_csv('../Data/grouped_invalsi.csv')

In [29]:
class MyLabelEncoder(LabelEncoder):

    def fit(self, y):
        y = column_or_1d(y, warn=True)
        self.classes_ = pd.Series(y).unique()
        return self
    
def compute_prop(data, col):
    return pd.crosstab(data.cod_scu_anonimo,data[col]).apply(lambda r: r/r.sum(), axis=1)

In [30]:
data

,cod_scu_anonimo,cod_cla_anonimo,cod_stu_anonimo,macrotipologia,sesso,mese,anno,luogo,eta,freq_asilo_nido,...,num_imm1,class_imm1,prop_imm1,num_imm2,class_imm2,prop_imm2,num_late,class_late,prop_late,public
0,29587,28127,20,ISTRUZIONE I CICLO,Maschio,Ottobre,2000,Italia (o,NaN,NaN,...,NaN,0,0.000000,NaN,0,0.000000,NaN,2,0.095238,1
1,29587,28127,10,ISTRUZIONE I CICLO,Maschio,Dicembre,2000,Italia (o,NaN,NaN,...,NaN,0,0.000000,NaN,0,0.000000,NaN,2,0.095238,1
2,29587,28127,4,ISTRUZIONE I CICLO,Femmina,Ottobre,2000,Italia (o,NaN,NaN,...,NaN,0,0.000000,NaN,0,0.000000,NaN,2,0.095238,1
3,29587,28127,9,ISTRUZIONE I CICLO,Femmina,Giugno,2000,Italia (o,NaN,NaN,...,NaN,0,0.000000,NaN,0,0.000000,NaN,2,0.095238,1
4,29587,28127,6,ISTRUZIONE I CICLO,Maschio,Giugno,2000,Italia (o,NaN,NaN,...,NaN,0,0.000000,NaN,0,0.000000,NaN,2,0.095238,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28140,31000,29586,28111,ISTRUZIONE I CICLO,Maschio,Maggio,2000,Italia (o,NaN,No,...,NaN,2,0.086957,NaN,1,0.043478,NaN,4,0.173913,0
28141,31000,29586,28123,ISTRUZIONE I CICLO,Femmina,Agosto,2000,Italia (o,NaN,No,...,NaN,2,0.086957,NaN,1,0.043478,NaN,4,0.173913,0
28142,31000,29586,28120,ISTRUZIONE I CICLO,Femmina,Maggio,2000,Italia (o,NaN,SÃ¬,...,NaN,2,0.086957,NaN,1,0.043478,NaN,4,0.173913,0
28143,31000,29586,28126,ISTRUZIONE I CICLO,Femmina,Luglio,1999,Unione Eur,8 anni,No,...,2.0,2,0.086957,NaN,1,0.043478,4.0,4,0.173913,0


In [31]:
student_data = pd.read_csv('../Data/cleaned_invalsi.csv')
data = pd.read_csv('../Data/grouped_invalsi.csv')

c:\users\gianm\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (88,89,90,91,92,93,187,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [32]:
aggr_dom = {'D1': 'opinione_invalsi',
           'D2': 'utilizzo_invalsi',
           'D4a': 'discussi_insegnanti',
           'D4f':'discussi_genitori',
           'D6': 'sodd_pon',
           'D11': 'attivita_preside',
           'D15': 'pressioni_genitori',
           'D16': 'coinvolgimento_genitori_prop',
           'D17': 'opinione_associazioni_genitori',
           'D18': 'coinvolgimento_genitori_eff',
           'D19': 'condotta_studenti',
           'D20': 'registro_elettronico',
           'D21': 'numero_plessi',
           'D22' :'infrastrutture',
           'D23': 'strumenti',
           'D24': 'preside_maschio',
           'D25': 'eta_preside'}

In [33]:
D1 = data.loc[:, data.columns.str.startswith('D1_')]
le = MyLabelEncoder()
le.fit(["Negativo", "Parzialmen", "Positivo"])
D1 = D1.replace('Non so/non', np.nan)
D1.dropna(inplace = True)
D1 = D1.apply(le.transform).mean(axis = 1)

In [34]:
D2 = data.loc[:, data.columns.str.startswith('D2_')]
le = MyLabelEncoder()
le.fit(["Per niente", "Poco", "Abbastanza", "Molto"])
D2 = D2.replace('Non so/non', np.nan)
D2.dropna(inplace = True)
D2 = D2.apply(le.transform).mean(axis = 1)

In [40]:
D4_a = (data.loc[:,'D4_a'] == 'SÃ¬').apply(int) #discussi con insegnanti
D4_f = (data.loc[:,'D4_f'] == 'SÃ¬').apply(int) #discussi con genitori

In [42]:
D6 = data.loc[:, data.columns.str.startswith('D6bis')]
le = MyLabelEncoder()
le.fit(["Per niente", "Poco", "Abbastanza", "Molto"])
D6 = D6.replace('Non so/non', np.nan)
D6.dropna(inplace = True)
D6 = D6.apply(le.transform).mean(axis = 1)

In [43]:
D11 = data.loc[:, data.columns.str.startswith('D11')]
le = MyLabelEncoder()
le.fit(["Mai o quas", "Qualche vo", "Spesso", "Sempre o q"])
D11 = D11.replace('Non so/non', np.nan)
D11.dropna(inplace = True)
D11 = D11.apply(le.transform).mean(axis = 1)

In [44]:
D14 = data.loc[:, data.columns.str.startswith('D14_')]

In [46]:
D15 = (data.loc[:,'D15'] == 'Câ€™Ã¨ una pr').apply(int) #pressioni genitori

In [47]:
D16 = data.loc[:, data.columns.str.startswith('D16')]
le = MyLabelEncoder()
le.fit(["Mai o quas", "Qualche vo", "Spesso", "Sempre o q"])
D16 = D16.replace('Non so/non', np.nan)
D16.dropna(inplace = True)
D16 = D16.apply(le.transform).mean(axis = 1)

In [49]:
D17 = data.loc[:, 'D17']
D17 = D17.replace('No',0)
D17 = D17.replace('SÃ¬, ma non',0)
D17 = D17.replace('SÃ¬ e giudi',1)
D17 = D17.replace('SÃ¬, ma cre',-1)

In [50]:
D18 = data.loc[:, data.columns.str.startswith('D18')]
le = MyLabelEncoder()
le.fit(["Per niente", "Poco", "Abbastanza", "Molto"])
D18 = D18.replace('Non so/non', np.nan)
D18.dropna(inplace = True)
D18 = D18.apply(le.transform).mean(axis = 1)

In [51]:
D19 = data.loc[:, data.columns.str.startswith('D19_')]
le = MyLabelEncoder()
le.fit(["Per niente", "Poco", "Abbastanza", "Molto"])
D19 = D19.replace('Non so/non', np.nan)
D19.dropna(inplace = True)
D19 = D19.apply(le.transform).mean(axis = 1)

In [53]:
D20 = ((data['D20'] == 'SÃ¬, in tut') | (data['D20'] == 'SÃ¬, solo i')).apply(int)

In [54]:
D21 = data['D21']

In [55]:
D22 = data.loc[:, data.columns.str.startswith('D22_')]
le = MyLabelEncoder()
le.fit(["Non presen", "Presente a"])
D22 = D22.replace('Non so/non', np.nan)
D22.dropna(inplace = True)
D22 = D22.apply(le.transform).mean(axis = 1)

In [56]:
D23 = data.loc[:, data.columns.str.startswith('D23_')]
le = MyLabelEncoder()
le.fit(["Del tutto","Presente m","Presente i"])
D23 = D23.replace('Non so/non', np.nan)
D23.dropna(inplace = True)
D23 = D23.apply(le.transform).mean(axis = 1)

In [57]:
D24 = (data.D24 == 'Maschio').apply(int) #preside maschio

In [58]:
D25 = 2014 - data.D25

In [59]:
temp = data.copy()#.drop('Unnamed: 0',axis=1)
temp = temp.join(D1.rename(aggr_dom['D1'])).drop(data.columns[data.columns.str.startswith('D1_')],axis=1)
temp = temp.join(D2.rename(aggr_dom['D2'])).drop(data.columns[data.columns.str.startswith('D2_')],axis=1)
temp = temp.join(D4_a.rename(aggr_dom['D4a'])).drop(data.columns[data.columns.str.startswith('D4_a')],axis=1)
temp = temp.join(D4_f.rename(aggr_dom['D4f'])).drop(data.columns[data.columns.str.startswith('D4_f')],axis=1)
temp = temp.join(D6.rename(aggr_dom['D6'])).drop(data.columns[data.columns.str.startswith('D6bis')],axis=1)
temp = temp.join(D11.rename(aggr_dom['D11'])).drop(data.columns[data.columns.str.startswith('D11')],axis=1)
temp = temp.join(D15.rename(aggr_dom['D15'])).drop(data.columns[data.columns.str.startswith('D15')],axis=1)
temp = temp.join(D16.rename(aggr_dom['D16'])).drop(data.columns[data.columns.str.startswith('D16')],axis=1)
temp = temp.join(D17.rename(aggr_dom['D17'])).drop(data.columns[data.columns.str.startswith('D17')],axis=1)
temp = temp.join(D18.rename(aggr_dom['D18'])).drop(data.columns[data.columns.str.startswith('D18')],axis=1)
temp = temp.join(D19.rename(aggr_dom['D19'])).drop(data.columns[data.columns.str.startswith('D19')],axis=1)
temp = temp.join(D20.rename(aggr_dom['D20'])).drop(data.columns[data.columns.str.startswith('D20')],axis=1)
temp = temp.join(D21.rename(aggr_dom['D21'])).drop(data.columns[data.columns.str.startswith('D21')],axis=1)
temp = temp.join(D22.rename(aggr_dom['D22'])).drop(data.columns[data.columns.str.startswith('D22')],axis=1)
temp = temp.join(D23.rename(aggr_dom['D23'])).drop(data.columns[data.columns.str.startswith('D23')],axis=1)
temp = temp.join(D24.rename(aggr_dom['D24'])).drop(data.columns[data.columns.str.startswith('D24')],axis=1)
temp = temp.join(D25.rename(aggr_dom['D25'])).drop(data.columns[data.columns.str.startswith('D25')],axis=1)
temp = temp.drop(temp.columns[temp.columns.str.startswith('D')], axis = 1)
le = MyLabelEncoder()
le.fit(["Area_non_P", "Area_Pon"])
temp['Pon'] = le.transform(temp['Pon'])
temp = temp.set_index('cod_scu_anonimo')
temp

,voto_scritto_ita,voto_scritto_mat,voto_scritto_ita_cont,voto_scritto_mat_cont,prov,Nome_reg,Pon,Fattore_correzione,attributo,TIPO_SCUOLA,...,coinvolgimento_genitori_prop,opinione_associazioni_genitori,coinvolgimento_genitori_eff,condotta_studenti,registro_elettronico,numero_plessi,infrastrutture,strumenti,preside_maschio,eta_preside
cod_scu_anonimo,,,,,,,,,,,,,,,,,,,,,
29587,NaN,NaN,NaN,NaN,PZ,Basilicata,0,0.987523,STATALE,ISTITUTO C,...,1.3750,0.0,1.111111,0.714286,0,7.0,0.545455,0.9,1,65.0
29588,8.0,9.0,8.0,9.0,CB,Molise,0,0.998281,NaN,NaN,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN
29589,9.0,9.0,9.0,9.0,CB,Molise,0,0.986064,STATALE,ISTITUTO C,...,NaN,0.0,NaN,1.142857,1,4.0,1.000000,1.8,1,59.0
29590,7.0,5.0,NaN,NaN,TN,Prov. Aut. Trento,0,0.993546,STATALE,ISTITUTO C,...,0.9375,1.0,1.277778,1.000000,0,10.0,0.818182,1.6,0,57.0
29591,NaN,NaN,NaN,NaN,TN,Prov. Aut. Trento,0,0.983432,NaN,NaN,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30996,NaN,NaN,NaN,NaN,CN,Piemonte,0,0.996134,STATALE,SCUOLA SEC,...,NaN,0.0,NaN,1.285714,0,4.0,0.818182,1.1,0,56.0
30997,7.0,6.0,7.0,6.0,CN,Piemonte,0,0.981022,STATALE,SCUOLA SEC,...,NaN,0.0,NaN,0.428571,0,3.0,0.818182,0.9,0,62.0
30998,NaN,NaN,NaN,NaN,AT,Piemonte,0,0.972717,STATALE,SCUOLA SEC,...,NaN,0.0,NaN,1.000000,1,2.0,0.818182,2.0,1,63.0


In [60]:
temp = temp.astype({'Pon': 'int32',
                   'public': 'int32',
                   'Nord': 'int32',
                   'Centro': 'int32',
                   'Sud': 'int32',})

In [61]:
management = temp[list(aggr_dom.values())]
management['sodd_pon'].fillna(-1, inplace = True)
management.dropna(inplace = True)
management

c:\users\gianm\appdata\local\programs\python\python38\lib\site-packages\pandas\core\series.py:4517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
<ipython-input-61-7e2d46b082a6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  management.dropna(inplace = True)


,opinione_invalsi,utilizzo_invalsi,discussi_insegnanti,discussi_genitori,sodd_pon,attivita_preside,pressioni_genitori,coinvolgimento_genitori_prop,opinione_associazioni_genitori,coinvolgimento_genitori_eff,condotta_studenti,registro_elettronico,numero_plessi,infrastrutture,strumenti,preside_maschio,eta_preside
cod_scu_anonimo,,,,,,,,,,,,,,,,,
29587,1.750,2.25,0,0,-1.0,1.857143,1,1.3750,0.0,1.111111,0.714286,0,7.0,0.545455,0.9,1,65.0
29590,1.375,3.00,0,1,-1.0,2.071429,0,0.9375,1.0,1.277778,1.000000,0,10.0,0.818182,1.6,0,57.0
29597,1.750,2.00,0,0,-1.0,2.000000,0,0.8125,1.0,1.111111,0.285714,0,9.0,0.909091,2.0,1,51.0
29599,1.875,2.25,1,0,-1.0,2.785714,0,1.4375,0.0,1.333333,1.142857,0,10.0,0.727273,1.7,1,57.0
29601,1.750,2.75,0,0,-1.0,2.071429,0,0.8125,0.0,1.000000,0.857143,0,10.0,0.454545,1.0,0,59.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30778,2.000,2.25,0,0,-1.0,1.571429,0,0.3750,0.0,1.055556,1.428571,1,3.0,0.818182,2.0,0,59.0
30779,2.000,2.25,0,0,-1.0,2.142857,1,0.7500,0.0,1.222222,0.428571,1,5.0,0.909091,1.7,0,56.0
30780,2.000,1.75,0,0,-1.0,2.500000,0,1.4375,1.0,1.833333,0.142857,1,6.0,0.818182,1.9,0,56.0


In [62]:
management = management.join(data.copy().set_index('cod_scu_anonimo'))
management

,opinione_invalsi,utilizzo_invalsi,discussi_insegnanti,discussi_genitori,sodd_pon,attivita_preside,pressioni_genitori,coinvolgimento_genitori_prop,opinione_associazioni_genitori,coinvolgimento_genitori_eff,...,prop_maschi,prop_laurea_padre,prop_laurea_madre,prop_diploma_padre,prop_diploma_madre,prop_padre_ita,prop_madre_ita,prop_padre_disocc,prop_madre_disocc,prop_bocciati
cod_scu_anonimo,,,,,,,,,,,,,,,,,,,,,
29587,1.750,2.25,0,0,-1.0,1.857143,1,1.3750,0.0,1.111111,...,0.619048,0.047619,0.000000,0.000000,0.047619,0.952381,0.904762,0.000000,0.000000,0.095238
29590,1.375,3.00,0,1,-1.0,2.071429,0,0.9375,1.0,1.277778,...,0.529412,0.058824,0.117647,0.235294,0.117647,1.000000,0.882353,0.000000,0.000000,0.000000
29597,1.750,2.00,0,0,-1.0,2.000000,0,0.8125,1.0,1.111111,...,0.461538,0.000000,0.000000,0.153846,0.384615,1.000000,0.923077,0.000000,0.000000,0.000000
29599,1.875,2.25,1,0,-1.0,2.785714,0,1.4375,0.0,1.333333,...,0.529412,0.058824,0.000000,0.235294,0.529412,1.000000,0.941176,0.176471,0.117647,0.117647
29601,1.750,2.75,0,0,-1.0,2.071429,0,0.8125,0.0,1.000000,...,0.409091,0.000000,0.000000,0.000000,0.181818,0.863636,0.636364,0.000000,0.000000,0.272727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30778,2.000,2.25,0,0,-1.0,1.571429,0,0.3750,0.0,1.055556,...,0.684211,0.052632,0.105263,0.157895,0.315789,0.684211,0.736842,0.157895,0.000000,0.210526
30779,2.000,2.25,0,0,-1.0,2.142857,1,0.7500,0.0,1.222222,...,0.521739,0.260870,0.217391,0.260870,0.478261,0.869565,0.739130,0.000000,0.000000,0.043478
30780,2.000,1.75,0,0,-1.0,2.500000,0,1.4375,1.0,1.833333,...,0.476190,0.190476,0.333333,0.142857,0.190476,0.714286,0.809524,0.000000,0.047619,0.190476


In [63]:
management = management.drop(management.columns[management.columns.str.startswith('D')], axis = 1)
le = MyLabelEncoder()
le.fit(["Area_non_P", "Area_Pon"])
management['Pon'] = le.transform(management['Pon'])
management

,opinione_invalsi,utilizzo_invalsi,discussi_insegnanti,discussi_genitori,sodd_pon,attivita_preside,pressioni_genitori,coinvolgimento_genitori_prop,opinione_associazioni_genitori,coinvolgimento_genitori_eff,...,prop_maschi,prop_laurea_padre,prop_laurea_madre,prop_diploma_padre,prop_diploma_madre,prop_padre_ita,prop_madre_ita,prop_padre_disocc,prop_madre_disocc,prop_bocciati
cod_scu_anonimo,,,,,,,,,,,,,,,,,,,,,
29587,1.750,2.25,0,0,-1.0,1.857143,1,1.3750,0.0,1.111111,...,0.619048,0.047619,0.000000,0.000000,0.047619,0.952381,0.904762,0.000000,0.000000,0.095238
29590,1.375,3.00,0,1,-1.0,2.071429,0,0.9375,1.0,1.277778,...,0.529412,0.058824,0.117647,0.235294,0.117647,1.000000,0.882353,0.000000,0.000000,0.000000
29597,1.750,2.00,0,0,-1.0,2.000000,0,0.8125,1.0,1.111111,...,0.461538,0.000000,0.000000,0.153846,0.384615,1.000000,0.923077,0.000000,0.000000,0.000000
29599,1.875,2.25,1,0,-1.0,2.785714,0,1.4375,0.0,1.333333,...,0.529412,0.058824,0.000000,0.235294,0.529412,1.000000,0.941176,0.176471,0.117647,0.117647
29601,1.750,2.75,0,0,-1.0,2.071429,0,0.8125,0.0,1.000000,...,0.409091,0.000000,0.000000,0.000000,0.181818,0.863636,0.636364,0.000000,0.000000,0.272727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30778,2.000,2.25,0,0,-1.0,1.571429,0,0.3750,0.0,1.055556,...,0.684211,0.052632,0.105263,0.157895,0.315789,0.684211,0.736842,0.157895,0.000000,0.210526
30779,2.000,2.25,0,0,-1.0,2.142857,1,0.7500,0.0,1.222222,...,0.521739,0.260870,0.217391,0.260870,0.478261,0.869565,0.739130,0.000000,0.000000,0.043478
30780,2.000,1.75,0,0,-1.0,2.500000,0,1.4375,1.0,1.833333,...,0.476190,0.190476,0.333333,0.142857,0.190476,0.714286,0.809524,0.000000,0.047619,0.190476


In [64]:
df = management.copy()
df = df.rename(columns=lambda x: re.sub("[(,)']","",x))
df = df.rename(columns=lambda x: re.sub(" ","_",x))
df = df.rename(columns=lambda x: re.sub("(,)","_",x))
df = df.rename(columns=lambda x: re.sub("<","",x))
df = df.rename(columns=lambda x: re.sub(">","",x))
df

,opinione_invalsi,utilizzo_invalsi,discussi_insegnanti,discussi_genitori,sodd_pon,attivita_preside,pressioni_genitori,coinvolgimento_genitori_prop,opinione_associazioni_genitori,coinvolgimento_genitori_eff,...,prop_maschi,prop_laurea_padre,prop_laurea_madre,prop_diploma_padre,prop_diploma_madre,prop_padre_ita,prop_madre_ita,prop_padre_disocc,prop_madre_disocc,prop_bocciati
cod_scu_anonimo,,,,,,,,,,,,,,,,,,,,,
29587,1.750,2.25,0,0,-1.0,1.857143,1,1.3750,0.0,1.111111,...,0.619048,0.047619,0.000000,0.000000,0.047619,0.952381,0.904762,0.000000,0.000000,0.095238
29590,1.375,3.00,0,1,-1.0,2.071429,0,0.9375,1.0,1.277778,...,0.529412,0.058824,0.117647,0.235294,0.117647,1.000000,0.882353,0.000000,0.000000,0.000000
29597,1.750,2.00,0,0,-1.0,2.000000,0,0.8125,1.0,1.111111,...,0.461538,0.000000,0.000000,0.153846,0.384615,1.000000,0.923077,0.000000,0.000000,0.000000
29599,1.875,2.25,1,0,-1.0,2.785714,0,1.4375,0.0,1.333333,...,0.529412,0.058824,0.000000,0.235294,0.529412,1.000000,0.941176,0.176471,0.117647,0.117647
29601,1.750,2.75,0,0,-1.0,2.071429,0,0.8125,0.0,1.000000,...,0.409091,0.000000,0.000000,0.000000,0.181818,0.863636,0.636364,0.000000,0.000000,0.272727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30778,2.000,2.25,0,0,-1.0,1.571429,0,0.3750,0.0,1.055556,...,0.684211,0.052632,0.105263,0.157895,0.315789,0.684211,0.736842,0.157895,0.000000,0.210526
30779,2.000,2.25,0,0,-1.0,2.142857,1,0.7500,0.0,1.222222,...,0.521739,0.260870,0.217391,0.260870,0.478261,0.869565,0.739130,0.000000,0.000000,0.043478
30780,2.000,1.75,0,0,-1.0,2.500000,0,1.4375,1.0,1.833333,...,0.476190,0.190476,0.333333,0.142857,0.190476,0.714286,0.809524,0.000000,0.047619,0.190476


In [65]:
list(df.columns)

['opinione_invalsi',
 'utilizzo_invalsi',
 'discussi_insegnanti',
 'discussi_genitori',
 'sodd_pon',
 'attivita_preside',
 'pressioni_genitori',
 'coinvolgimento_genitori_prop',
 'opinione_associazioni_genitori',
 'coinvolgimento_genitori_eff',
 'condotta_studenti',
 'registro_elettronico',
 'numero_plessi',
 'infrastrutture',
 'strumenti',
 'preside_maschio',
 'eta_preside',
 'voto_scritto_ita',
 'voto_scritto_mat',
 'voto_scritto_ita_cont',
 'voto_scritto_mat_cont',
 'prov',
 'Nome_reg',
 'Pon',
 'Fattore_correzione',
 'attributo',
 'TIPO_SCUOLA',
 'Cod_reg',
 'Areageo_3',
 'Areageo_4',
 'Areageo_5',
 'Areageo_5_Istat',
 'statale',
 'paritaria',
 'Nord',
 'Centro',
 'Sud',
 'school_score_ita',
 'school_score_mate',
 'school_escs',
 'class_tot',
 'class_fem',
 'prop_fem',
 'class_imm1',
 'prop_imm1',
 'class_imm2',
 'prop_imm2',
 'class_late',
 'prop_late',
 'public',
 'voto_orale_ita_mean',
 'voto_orale_ita_std',
 'voto_orale_ita_skew',
 'voto_orale_ita_min',
 'voto_orale_ita_max',
 

In [66]:
df.to_csv('../Data/gppp.csv')